# Workflow State Store

> Server-side workflow state storage implementations

In [ ]:
#| default_exp core.state_store

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import uuid
from typing import Dict, Any, Optional

from cjm_fasthtml_interactions.patterns.step_flow import WorkflowStateStore

## Session ID Extraction

The state store needs a way to identify individual users/sessions. We store a small UUID in the session object itself (the only thing we store there) and use it as a key for server-side storage.

In [ ]:
#| export
def get_session_id(
    sess: Any,  # FastHTML session object
    key: str = "_workflow_session_id"  # Session key for storing the ID
) -> str:  # Stable session identifier
    """Get or create a stable session identifier."""
    if key not in sess:
        sess[key] = str(uuid.uuid4())
    return sess[key]

## InMemoryWorkflowStateStore

A simple in-memory implementation of `WorkflowStateStore` suitable for development and testing. State is lost when the server restarts.

For production use, this can be replaced with a file-based or database-backed implementation that follows the same protocol.

In [ ]:
#| export
class InMemoryWorkflowStateStore:
    """In-memory workflow state storage for development and testing."""
    
    def __init__(self):
        """Initialize empty state storage."""
        self._current_steps: Dict[str, str] = {}  # {flow_id:session_id -> step_id}
        self._states: Dict[str, Dict[str, Any]] = {}  # {flow_id:session_id -> state}
    
    def _make_key(self,
                  flow_id: str,  # Workflow identifier
                  sess: Any  # FastHTML session object
                 ) -> str:  # Composite key for storage
        """Create composite key from flow_id and session."""
        session_id = get_session_id(sess)
        return f"{flow_id}:{session_id}"
    
    def get_current_step(self,
                         flow_id: str,  # Workflow identifier
                         sess: Any  # FastHTML session object
                        ) -> Optional[str]:  # Current step ID or None
        """Get current step ID for a workflow."""
        key = self._make_key(flow_id, sess)
        return self._current_steps.get(key)
    
    def set_current_step(self,
                         flow_id: str,  # Workflow identifier
                         sess: Any,  # FastHTML session object
                         step_id: str  # Step ID to set as current
                        ) -> None:
        """Set current step ID for a workflow."""
        key = self._make_key(flow_id, sess)
        self._current_steps[key] = step_id
    
    def get_state(self,
                  flow_id: str,  # Workflow identifier
                  sess: Any  # FastHTML session object
                 ) -> Dict[str, Any]:  # Workflow state dictionary
        """Get all workflow state."""
        key = self._make_key(flow_id, sess)
        return self._states.get(key, {}).copy()
    
    def update_state(self,
                     flow_id: str,  # Workflow identifier
                     sess: Any,  # FastHTML session object
                     updates: Dict[str, Any]  # State updates to apply
                    ) -> None:
        """Update workflow state with new values."""
        key = self._make_key(flow_id, sess)
        if key not in self._states:
            self._states[key] = {}
        self._states[key].update(updates)
    
    def clear_state(self,
                    flow_id: str,  # Workflow identifier
                    sess: Any  # FastHTML session object
                   ) -> None:
        """Clear all workflow state."""
        key = self._make_key(flow_id, sess)
        self._current_steps.pop(key, None)
        self._states.pop(key, None)

## Usage Example

In [ ]:
# Create a store instance
store = InMemoryWorkflowStateStore()

# Mock session object (in real usage, this comes from FastHTML)
mock_sess = {}

# Store and retrieve state
store.set_current_step("my_workflow", mock_sess, "step_1")
print(f"Current step: {store.get_current_step('my_workflow', mock_sess)}")

store.update_state("my_workflow", mock_sess, {"name": "Alice", "email": "alice@example.com"})
print(f"State: {store.get_state('my_workflow', mock_sess)}")

# Session only contains the session ID
print(f"Session contents: {mock_sess}")

Current step: step_1
State: {'name': 'Alice', 'email': 'alice@example.com'}
Session contents: {'_workflow_session_id': '760d1d51-ab0a-44e1-be78-e346fb7d6874'}


In [ ]:
# Verify it satisfies the protocol
assert isinstance(store, WorkflowStateStore), "InMemoryWorkflowStateStore must implement WorkflowStateStore protocol"
print("Protocol check passed!")

Protocol check passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()